In [2]:
### Notebook for GSP import into x-nat central
import pyxnat
from pyxnat import *
central = Interface(server="https://central.xnat.org", user= '<user-name>', password= '<password>')
#mid_xnat = Interface(server="https://mind-xnat.nyspi.org", user= '<user-name>', password= '<password>')
central.save_config('./central.cfg')
#mind_xnat.save_config('./mind_xnat.cfg')

In [3]:
metadata_file='metadata.csv'

data_path='./subjects/'


In [4]:
import pandas as pd
import numpy as np
import glob
import os
import numpy
scanner_id = {}
metadata = pd.read_csv('metadata.csv')
i = 1

subjects = {}
for index, row in metadata.iterrows():
    if row['Age_Bin'] and row['Sex'] and row['Scanner_Bin'] :
        
        sub_id = row['Subject_ID'].split('_')[0]
        if row['Subject_Rescan_ID']:
            no_scans =2
        else:
            no_scans=1
        s= sub_id
        if row['Age_Bin']>0 and row['Age_Bin']<120 :
            
            subjects[s]= {'id':sub_id}
            subjects[s]['Age']= str(int(row['Age_Bin']))
            if (row['Sex'])=='M':
                
                subjects[s]['Sex']= 'Male'
            else:
                subjects[s]['Sex']= 'Female'
            subjects[s]['Scanner']= row['Scanner_Bin']
            subjects[s]['no_scans']= no_scans
            
            
            

In [5]:
import os
import glob
project_name = 'TEST_MIND'
xnat=pyxnat.Interface(config='./central.cfg')
#xnat=pyxnat.Interface(config='./mind_xnat.cfg')
cwd = os.getcwd()
print cwd
xnat.inspect.structure()
project=xnat.select.project(project_name)

if not project.exists():
    project.insert()
for path,j,y in os.walk(data_path):
    exp='.nii.gz'
    
    files = os.listdir(path)
    for file in files:
        if file.endswith(exp):
            sub_id, session_no, scan, scan_no, scan_type=file.split('_')
            if sub_id in subjects.keys():    
            #print subjects[sub_id]['id']
                sub=project.subject(subjects[sub_id]['id'])
                #print sub.get()
                if not sub.exists():
                    sub.insert()
                if session_no == 'Ses1':
                    mr = '_MR1'
                else:
                    mr = '_MR2'
                session_label = project_name + '_' + subjects[sub_id]['id'] + mr
                print session_label
                #session_label="%s_%s_MR1" % (project_name,subjects[sub_id]['id'])
                print type(subjects[sub_id]['Age'])
                sub.attrs.set('age',subjects[sub_id]['Age'])
                #sub.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/age',subjects[sub_id]['Age'])
                sub.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/gender',subjects[sub_id]['Sex'])
                experiment=sub.experiment(session_label)

                if not experiment.exists():
                    experiment.create()

                #experiment.attrs.set('xnat:experimentData/date',subjects[sub_id]['date'])
                experiment.attrs.set('xnat:imageSessionData/scanner','Siemens 3T Tim Trio')

                experiment.attrs.set('xnat:imageSessionData/fieldStrength','3T')


                print path
                #configfiles = os.listdir(path)
                actual_path = path + '/' + file
                if scan_type== 'ANAT1.nii.gz' or scan_type=='ANAT2.nii.gz':
                    img_reorient='anonymized_reorient.nii.gz'
                    print actual_path
                    cmd_reorient='fslreorient2std '+  actual_path + ' ' + img_reorient
                    print cmd_reorient
                    os.system(cmd_reorient)

                # Now insert the reoriented image
                # experiment.scan('1').resource('NIFTI').file(os.path.basename(scans_filtered[sub])).insert(
                    name = 'T1_'+scan_no
                    experiment.scan(name).resource('NIFTI').file(os.path.basename(img_reorient)).insert(
                    img_reorient,
                    content='T1',
                    overwrite=True,
                    #Note = 
                    type='T1',  # May need to change this to SPGR, MPRAGE, FLAIR etc?
                    format='NIFTI')

                    # Create ORIGINAL and SNAPSHOT to view the images in the web browser
                    img_png='tmp.png'
                    img_gif='tmp.gif'
                    img_thumb='tmp_thumb.gif'
                    cmd_create_slices='slicer ' + img_reorient + ' -S 8 800 ' + img_png
                    cmd_convert_gif='convert ' + img_png + ' ' + img_gif
                    cmd_convert_thumb='convert ' + img_png + ' -resize 240x240 ' + img_thumb

                    os.system(cmd_create_slices)
                    os.system(cmd_convert_gif)
                    os.system(cmd_convert_thumb)

                    experiment.scan(name).resource('SNAPSHOTS').file(os.path.basename(img_gif)).insert(
                    img_gif,
                    content='ORIGINAL',
                    overwrite=True,
                    format='GIF')

                    experiment.scan(name).resource('SNAPSHOTS').file(os.path.basename(img_thumb)).insert(
                    img_thumb,
                    content='THUMBNAIL',
                    overwrite=True,
                    format='GIF')
                elif scan_type== 'BOLD1.nii.gz' or scan_type=='BOLD2.nii.gz':
                    img_reorient='anonymized_reorient.nii.gz'
                    print actual_path
                    cmd_reorient='fslreorient2std '+  actual_path + ' ' + img_reorient
                    os.system(cmd_reorient)
                    name = 'func_rest_'+scan_no
                # Now insert the reoriented image
                # experiment.scan('1').resource('NIFTI').file(os.path.basename(scans_filtered[sub])).insert(
                    experiment.scan(name).resource('NIFTI').file(os.path.basename(img_reorient)).insert(
                    img_reorient,
                    content='func_rest',
                    overwrite=True,
                    #Note = 
                    type='func_rest',  # May need to change this to SPGR, MPRAGE, FLAIR etc?
                    format='NIFTI')

                    # Create ORIGINAL and SNAPSHOT to view the images in the web browser
                    img_png='tmp.png'
                    print img_png
                    img_gif='tmp.gif'
                    
                    img_thumb='tmp_thumb.gif'
                    cmd_create_slices='slicer ' + img_reorient + ' -S 2 400 ' + img_png
                    cmd_convert_gif='convert ' + img_png + ' ' + img_gif
                    cmd_convert_thumb='convert ' + img_png + ' -resize 240x240 ' + img_thumb

                    os.system(cmd_create_slices)
                    os.system(cmd_convert_gif)
                    os.system(cmd_convert_thumb)

                    experiment.scan(name).resource('SNAPSHOTS').file(os.path.basename(img_gif)).insert(
                    img_gif,
                    content='ORIGINAL',
                    overwrite=True,
                    format='GIF')

                    experiment.scan(name).resource('SNAPSHOTS').file(os.path.basename(img_thumb)).insert(
                    img_thumb,
                    content='THUMBNAIL',
                    overwrite=True,
                    format='GIF')

     
            print "ok"

 


       
        
            

/Users/abc/Desktop/spring2017/Neuroimaging/GSP/GSP_import
- PROJECTS
    +SUBJECTS
        +EXPERIMENTS
            +ASSESSORS
                +RESOURCES
                    +FILES
                +IN_RESOURCES
                    +FILES
                +OUT_RESOURCES
                    +FILES
            +RECONSTRUCTIONS
                +IN_RESOURCES
                    +FILES
                +OUT_RESOURCES
                    +FILES
            +SCANS
                +RESOURCES
                    +FILES
            +RESOURCES
                +FILES
        +RESOURCES
            +FILES
    +RESOURCES
        +FILES
TEST_MIND_Sub0043_MR1
<type 'str'>
./subjects/Sub0043_Ses1
./subjects/Sub0043_Ses1/Sub0043_Ses1_Scan_01_ANAT1.nii.gz
fslreorient2std ./subjects/Sub0043_Ses1/Sub0043_Ses1_Scan_01_ANAT1.nii.gz anonymized_reorient.nii.gz
ok
TEST_MIND_Sub0043_MR1
<type 'str'>
./subjects/Sub0043_Ses1
./subjects/Sub0043_Ses1/Sub0043_Ses1_Scan_02_BOLD1.nii.gz
tmp.png
ok
TEST_MIND_Sub0043_MR2
<t